<a href="https://colab.research.google.com/github/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/blob/master/Stocks_Sharpe_Ratios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
!pip install bokeh

from bokeh.io import curdoc

#bokeh.sampledata.download()
#from bokeh.sampledata.stocks import AAPL

# Perform necessary imports
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Band
from bokeh.models import NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

In [0]:
# Reading in the data
stock_data = pd.read_csv('https://raw.githubusercontent.com/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/master/stock_data.csv',
                         parse_dates=['Date'],
                         index_col='Date').dropna()

benchmark_data = pd.read_csv('https://raw.githubusercontent.com/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/master/benchmark_data.csv',
                             parse_dates=['Date'],
                             index_col='Date').dropna()


stock_returns = stock_data.pct_change()

sp_returns = benchmark_data['S&P 500'].pct_change()




In [0]:
def bollinger_bands(series, window=20, stds=2):
    
    #rolling metrics
    sma = series.rolling(window).mean()
    std = series.rolling(window).std()

    #bounds
    upper = sma + std * stds
    lower = sma - std * stds

    #yields
    return pd.DataFrame(index=series.index, data={
        'upper': upper,
        'mid': sma,
        'lower': lower
    })


In [0]:
 df_amazon = bollinger_bands(stock_data.Amazon)
 df_facebook = bollinger_bands(stock_data.Facebook)
 df_snp = bollinger_bands(benchmark_data['S&P 500'])

In [14]:
stock_data.info()
display(stock_data.shape)
display(stock_data.head())
display(stock_data.describe())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252 entries, 2016-01-04 to 2016-12-30
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Amazon    252 non-null    float64
 1   Facebook  252 non-null    float64
dtypes: float64(2)
memory usage: 5.9 KB


(252, 2)

,Amazon,Facebook
Date,,
2016-01-04,636.989990,102.220001
2016-01-05,633.789978,102.730003
2016-01-06,632.650024,102.970001
2016-01-07,607.940002,97.919998
2016-01-08,607.049988,97.330002


,Amazon,Facebook
count,252.000000,252.000000
mean,699.523135,117.035873
std,92.362312,8.899858
min,482.070007,94.160004
25%,606.929993,112.202499
50%,727.875000,117.765000
75%,767.882492,123.902503
max,844.359985,133.279999


In [0]:
def datetime(x):
    return np.array(x, dtype=np.datetime64)

source = ColumnDataSource(data={
    'x' : datetime(stock_data.index),
    #amazon data
    'Amazon' : stock_data.Amazon,
    'Amazon_l' : df_amazon.lower,
    'Amazon_m' : df_amazon.mid,
    'Amazon_u' : df_amazon.upper,
    'Amazon_r' : stock_returns.Amazon,

    #facebook data
    'Facebook' : stock_data.Facebook,
    'Facebook_l' : df_facebook.lower,
    'Facebook_m' : df_facebook.mid,
    'Facebook_u' : df_facebook.upper,
    'Facebook_r' : stock_returns.Facebook
})

source_snp = ColumnDataSource(data={
    'x' : datetime(benchmark_data.index),

    #S&P 500 data
    'y' : benchmark_data['S&P 500'],
    'l' : df_snp.lower,
    'm' : df_snp.mid,
    'u' : df_snp.upper,
    'r' : sp_returns
})

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

### Amazon

>#### Stock

In [0]:
p1 = figure(x_axis_type='datetime',
           title='Amazon Stock Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700,
           )
p1.yaxis.formatter = NumeralTickFormatter(format="$0.00")

# change just some things about the x-grid
p1.xgrid.grid_line_color = None

# change just some things about the y-grid
p1.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p1.background_fill_color="white"

#Amazon line plot
p1.line('x', 'Amazon', color='#FF9900', line_width=2, source=source)
p1.line('x', 'Amazon_m', color='black', source=source)
cr1 = p1.circle('x', 'Amazon', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

band1 = Band(base='x', lower='Amazon_l', upper='Amazon_u',
             source=source,
             fill_alpha=0.08, fill_color='black',
             line_width=1, line_color='black',
             level='underlay')

p1.add_layout(band1)

p1.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        #( 'Value ($)',  '$@{Facebook}' ),
        ( 'Value ($)',  '$@{Amazon}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr1], 
    #show_arrow=True
))

output_notebook()
#show(p1)


> #### Daily %

In [0]:
r1 = figure(x_axis_type='datetime',
           title='Amazon Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r1.yaxis.formatter = NumeralTickFormatter(format="0.00%")

# change just some things about the x-grid
r1.xgrid.grid_line_color = None

# change just some things about the y-grid
r1.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

r1.background_fill_color="white"

#bands
band_1r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.1, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - 2*stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + 2*stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.067, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - 3*stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + 3*stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.035, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_3r1)

#Amazon line plot
r1.line('x', 'Amazon_r', color='#FF9900', line_width=3, source=source)

cr_r1 = r1.circle('x', 'Amazon_r', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

r1.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        #( 'Value ($)',  '$@{Facebook}' ),
        ( '% Change',  '@{Amazon_r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r1], 
    #show_arrow=True
))

output_notebook()
show(r1)

### Facebook

> #### Stock

In [0]:
p2 = figure(x_axis_type='datetime',
           title='Facebook Stock Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
p2.yaxis.formatter = NumeralTickFormatter(format="$0.00")

# change just some things about the x-grid
p2.xgrid.grid_line_color = None

# change just some things about the y-grid
p2.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p2.background_fill_color="white"

p2.border_fill_color = "white"
p2.min_border_left = 80

band2 = Band(base='x', lower='Facebook_l', upper='Facebook_u',
             source=source,
             fill_alpha=0.3, fill_color='#6699ff',
             line_width=1, line_color='#6699ff',
             level='underlay')

p2.add_layout(band2)

#Facebook line plot
p2.line('x', 'Facebook', color='#4267B2', line_width=2, source=source)

cr2 = p2.triangle('x', 'Facebook', source=source, size=2,
              fill_color="grey", hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

#p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='vline'))

p2.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        ( 'Value ($)',  '$@{Facebook}' ),
        #( 'Value ($)',  '$@{Amazon}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr2], 
    #show_arrow=True
))


output_notebook()
#show(p2)





> #### Daily %

In [0]:
r2 = figure(x_axis_type='datetime',
           title='Facebook Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r2.yaxis.formatter = NumeralTickFormatter(format="0.00%")

# change just some things about the x-grid
r2.xgrid.grid_line_color = None

# change just some things about the y-grid
r2.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

r2.background_fill_color="white"

#bands
band_1r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.1, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - 2*stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + 2*stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.067, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - 3*stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + 3*stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.035, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_3r1)


r2.line('x', 'Facebook_r', color='#4267B2', line_width=3, source=source)

cr_r2 = r2.circle('x', 'Facebook_r', source=source, size=3,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

r2.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        #( 'Value ($)',  '$@{Facebook}' ),
        ( '% Change',  '@{Facebook_r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r2], 
    #show_arrow=True
))

output_notebook()
show(r2)


### S&P 500

> #### Stock

In [0]:
p3 = figure(x_axis_type='datetime',
           title='S&P 500 Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
p3.yaxis.formatter = NumeralTickFormatter(format="$0.00")

# change just some things about the x-grid
p3.xgrid.grid_line_color = None

# change just some things about the y-grid
p3.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p3.background_fill_color="white"

p3.border_fill_color = "white"
p3.min_border_left = 80

band3 = Band(base='x', lower='l', upper='u',
             source=source_snp,
             fill_alpha=0.3, fill_color='#b3b3ff',
             line_width=1, line_color='#b3b3ff',
             level='underlay')

p3.add_layout(band3)


p3.line('x', 'y', color='#8080ff', line_width=2, source=source_snp)

cr3 = p3.circle('x', 'y', source=source_snp, size=2,
              fill_color="grey", hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

#p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='vline'))

p3.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        ( 'Value ($)',  '$@{y}{0.000}' ),
        #( 'Value ($)',  '$@{Amazon}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr3], 
    #show_arrow=True
))


output_notebook()
#show(p3)




> #### Daily %

In [0]:
r3 = figure(x_axis_type='datetime',
           title='S&P 500 Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r3.yaxis.formatter = NumeralTickFormatter(format="0.00%")

# change just some things about the x-grid
r3.xgrid.grid_line_color = None

# change just some things about the y-grid
r3.ygrid.grid_line_alpha = 0.5

#p.ygrid.grid_line_dash = [6, 4]
r3.background_fill_color="white"

#4d4dff

#bands
band_1r1 = Band(base='x',
               lower=sp_returns.mean() - sp_returns.std(),
               upper=sp_returns.mean() + sp_returns.std(),
               source=source_snp,
               fill_alpha=0.1, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=sp_returns.mean() - 2*sp_returns.std(),
               upper=sp_returns.mean() + 2*sp_returns.std(),
               source=source_snp,
               fill_alpha=0.1, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=sp_returns.mean() - 3*sp_returns.std(),
               upper=sp_returns.mean() + 3*sp_returns.std(),
               source=source_snp,
               fill_alpha=0.1, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_3r1)


r3.line('x', 'r', color='#8080ff', line_width=3, source=source_snp)

cr_r3 = r3.circle('x', 'r', source=source_snp, size=3,
              fill_color=None, hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

r3.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@x{%F}'            ),
        #( 'Value ($)',  '$@{Facebook}' ),
        ( '% Change',  '@{r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r3], 
    #show_arrow=True
))

output_notebook()
show(r3)

### Assembling grid plot

In [0]:


tab_1 = gridplot([[p1, r1], [p2, r2], [p3, r3]], plot_width=650, plot_height=280)  # open a browser

In [43]:
from bokeh.models.widgets import Panel
from bokeh.models.widgets import Tabs
from bokeh.plotting import figure, output_file, save

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

tab1 = Panel(child=tab_1, title='gibberish')

layout = Tabs(tabs=[tab1])

# Specify the name of the output_file and show the result
output_file('tabs.html', mode='inline')
show(layout)



<IPython.core.display.Javascript object>

TypeError: ignored

In [0]:
excess_returns = stock_returns.sub(sp_returns, axis=0)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ImportError: ignored

ERROR: Could not find a version that satisfies the requirement bokeh.mpl (from versions: none)
ERROR: No matching distribution found for bokeh.mpl


In [36]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 2.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [244]:
import numpy as np

from bokeh.plotting import figure, output_file, show

N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

p.scatter(x, y, radius=radii,
          fill_color=colors, fill_alpha=0.6,
          line_color=None)

output_file("color_scatter.html", title="color_scatter.py example")

show(p)  # open a browser

In [0]:
p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
        ( 'volume', '@volume{0.00 a}'      ),
    ],

    formatters={
        'date'      : 'datetime', # use 'datetime' formatter for 'date' field
        'adj close' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
))